In [ ]:
import ast
import re
import requests
from datetime import datetime
from io import StringIO
from unidecode import unidecode

import pandas as pd
from bs4 import BeautifulSoup

from pocketknife.database import (connect_database, read_from_database)

from success_prediction.config import RAW_DATA_DIR, EXTERNAL_DATA_DIR, PROCESSED_DATA_DIR

## LOAD DATA FROM ZEFIX TO ATTACH THE EHRAID AND UID

In [ ]:
with connect_database() as con:
    df = read_from_database(
        con,
        query="""
            SELECT * FROM zefix.base base WHERE
                NOT base.is_branch
                AND NOT base.legal_form_id IN (9, 11, 18, 19)
                AND LOWER(base.name) NOT LIKE '%zweigniederlassung%'
                AND LOWER(base.name) NOT LIKE '%succursale%'
                AND (base.delete_date >= '2016-01-01' OR base.delete_date IS NULL);
        """
    )

with connect_database() as con:
    df_previous_names = read_from_database(con, "SELECT * FROM zefix.old_names")

In [ ]:
# Combine base data with previous names for the matching

df_grouped = df_previous_names.groupby('ehraid')['name'].apply(lambda x: [el for el in x]).reset_index()
df_grouped = df_grouped.rename(columns={'name': 'old_names'})

df = df.merge(df_grouped, on=['ehraid'], how='left')
df = df.rename(columns={'legal_seat': 'city'})

df['all_names'] = [old_names + [current_name] if isinstance(old_names, list) else [current_name] for old_names, current_name in zip(df['old_names'], df['name'])]
df = df.explode(column='all_names')

In [ ]:
# Normalize the firm names before matching

df['firm_name_norm'] = (df['all_names']
    .str.lower()
    .str.replace('in liquidation', '')
    .str.replace('en liquidation', '')
    .str.replace('in liquidazione', '')
    .str.strip()
)
df['firm_name_norm'] = df['firm_name_norm'].apply(unidecode).apply(lambda x: ' '.join([el for el in x.split()]))
df['firm_name_clean'] = (df['firm_name_norm']
    .str.replace('.', '', regex=False)
    .str.replace(r' ag$', '', regex=True)
    .str.replace(r' sa$', '', regex=True)
    .str.replace(r' gmbh$', '', regex=True)
    .str.replace(r' sarl$', '', regex=True)
    .str.replace(r' ltd$', '', regex=True)
    .str.replace(r' llc$', '', regex=True)
    .str.strip()
)
df['firm_name_clean'] = df['firm_name_clean'].str.replace(' ', '', regex=False)

df = df.drop_duplicates(subset=['firm_name_norm', 'uid'], keep='first')

## GET FUNDING DATA FROM STARTUP.CH

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

cookies = {
    "CFID": "22868597",
    "CFTOKEN": "c03f47d2f643a0ec-FFC1597B-0A9F-6C03-1550C3A2498E7C58",
    "CFGLOBALS": "urltoken%3DCFID%23%3D22868597%26CFTOKEN%23%3Dc03f47d2f643a0ec%2DFFC1597B%2D0A9F%2D6C03%2D1550C3A2498E7C58%23lastvisit%3D%7Bts%20%272025%2D04%2D14%2009%3A54%3A26%27%7D%23hitcount%3D114%23timecreated%3D%7Bts%20%272025%2D01%2D13%2016%3A23%3A03%27%7D%23cftoken%3Dc03f47d2f643a0ec%2DFFC1597B%2D0A9F%2D6C03%2D1550C3A2498E7C58%23cfid%3D22868597%23"
}

dfs = []
for list_idx, page_idx in zip(range(1, 100_000, 10), range(1, 10_000)):
    url = f"https://www.startup.ch/index.cfm?cfid=22868597&cftoken=c03f47d2f643a0ec-FFC1597B-0A9F-6C03-1550C3A2498E7C58&bericht_id=10000&start_liste_10000={list_idx}&bericht_seite_10000={page_idx}&page=137906#fgBerichtAnker_10000"
    response = requests.get(url, headers=headers, cookies=cookies)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        table_div = soup.find("div", class_="founding-table")
        table = table_div.find("table") if table_div else None

        if table:
            dfs.append(pd.read_html(StringIO(str(table)))[0])
        else:
            print(f"Table not found or not accessible for {url}")
    else:
        print(f"Page not found: {url}")
        break

In [ ]:
combined = pd.concat(dfs, axis=0).reset_index(drop=True)

combined['DATE'] = combined['DATE'].str.replace('1900', '2019')
combined['DATE'] = pd.to_datetime(combined['DATE'], format='%d.%m.%Y')
combined = combined.dropna(subset='DATE')
combined = combined.set_index('DATE')
combined = combined.sort_index()
filtered = combined['2016-01-01':]
filtered = filtered.drop_duplicates().reset_index()
filtered.columns = ['date', 'firm_name', 'amount_chf', 'type', 'location']
filtered.to_csv(EXTERNAL_DATA_DIR / 'startup-ch_funding.csv', index=False)

filtered['firm_name_norm'] = filtered['firm_name'].str.lower().apply(lambda x: re.sub(r'\([^)]*\)', '', x))
filtered['firm_name_clean'] = filtered['firm_name_norm']

In [ ]:
# Merge startup.ch data with zefix data
merged = filtered.merge(df[['firm_name_norm', 'ehraid', 'uid', 'legal_seat']], on=['firm_name_norm'], how='left')

In [ ]:
matched = merged[~merged.ehraid.isna()]
missing = merged[merged.ehraid.isna()].drop(columns=['ehraid', 'uid'])
missing = missing.merge(df[['firm_name_clean', 'ehraid', 'uid']], on=['firm_name_clean'], how='left')

matched = pd.concat([matched, missing[~missing.ehraid.isna()]])
missing = missing[missing.ehraid.isna()].sort_values(['firm_name'])

duplicates = matched[matched.duplicated(subset=['date', 'firm_name', 'type', 'location'], keep=False)].sort_values(['firm_name', 'ehraid'])

unique_matched = matched[~matched.duplicated(subset=['date', 'firm_name', 'type', 'location'], keep=False)]

In [ ]:
unique_matched.to_csv(EXTERNAL_DATA_DIR / 'uniquely_matched.csv', index=False)
duplicates.to_csv(EXTERNAL_DATA_DIR / 'duplicates.csv', index=False)
missing.to_csv(EXTERNAL_DATA_DIR / 'missing.csv', index=False)

## LOAD SCRAPED INNOSUISSE DATA

In [ ]:
inno_df = pd.read_csv(RAW_DATA_DIR / 'funding_data' / 'innosuisse_grants.csv')

for col in ['contact_person', 'scientific_management', 'implementation_partner']:
    inno_df[col] = inno_df[col].apply(ast.literal_eval)

for col in ['start_date', 'end_date']:
    inno_df[col] = pd.to_datetime(inno_df[col])

inno_df = inno_df[(inno_df['implementation_partner'].apply(bool)) & (inno_df['start_date'] >= datetime(2016, 1, 1))]

inno_df = inno_df.explode(column=['implementation_partner'])

inno_df['uid'] = inno_df['implementation_partner'].apply(lambda x: x.get('uid_1'))
inno_df['canton'] = inno_df['implementation_partner'].apply(lambda x: x.get('canton'))
inno_df['city'] = inno_df['implementation_partner'].apply(lambda x: x.get('city'))
inno_df['zip_code'] = inno_df['implementation_partner'].apply(lambda x: x.get('zip_code'))
inno_df['street'] = inno_df['implementation_partner'].apply(lambda x: x.get('street'))

inno_df['firm_name_original'] = inno_df['implementation_partner'].apply(lambda x: x.get('company_1'))

inno_df['firm_name_norm'] = inno_df['implementation_partner'].apply(lambda x: x.get('company_1'))
inno_df['firm_name_combined'] = inno_df['implementation_partner'].apply(lambda x: ' '.join([x.get(col) for col in ['company_1', 'company_2', 'company_3', 'company_4'] if isinstance(x.get(col), str)]))

inno_df['firm_name_norm'] = inno_df['firm_name_norm'].str.lower().apply(lambda x: ' '.join([el for el in x.split()]))
inno_df['firm_name_norm'] = inno_df['firm_name_norm'].apply(unidecode)

inno_df['firm_name_combined'] = inno_df['firm_name_combined'].str.lower().apply(lambda x: ' '.join([el for el in x.split()]))

inno_df['firm_name_clean'] = (inno_df['firm_name_norm']
    .str.replace('.', '', regex=False)
    .str.replace(r' ag$', '', regex=True)
    .str.replace(r' sa$', '', regex=True)
    .str.replace(r' gmbh$', '', regex=True)
    .str.replace(r' sarl$', '', regex=True)
    .str.replace(r' ltd$', '', regex=True)
    .str.strip()
)
inno_df['firm_name_clean'] = inno_df['firm_name_clean'].str.replace(' ', '', regex=False)

In [ ]:
# Split into entries with UID and entries without
inno_df_matched = inno_df[~inno_df.uid.isna()].copy()
inno_df_missing = inno_df[inno_df.uid.isna()].copy()

In [ ]:
# First match on firm_name and city to reduce duplicates
inno_df_missing = inno_df_missing.drop(columns=['uid'])
inno_df_missing = inno_df_missing.merge(df[['firm_name_norm', 'city', 'uid']], on=['firm_name_norm', 'city'], how='left')

inno_df_matched = pd.concat([inno_df_matched, inno_df_missing[~inno_df_missing.uid.isna()]])
inno_df_missing = inno_df_missing[inno_df_missing.uid.isna()].sort_values(['firm_name_norm'])

In [ ]:
# Then only match remaining only on firm_name
inno_df_missing = inno_df_missing.drop(columns=['uid'])
inno_df_missing = inno_df_missing.merge(df[['firm_name_norm', 'uid']], on=['firm_name_norm'], how='left')

inno_df_matched = pd.concat([inno_df_matched, inno_df_missing[~inno_df_missing.uid.isna()]])
inno_df_missing = inno_df_missing[inno_df_missing.uid.isna()].sort_values(['firm_name_norm'])

In [ ]:
# Then only match remaining on firm name without common company type indicators (e.g. AG, GmbH, SA, Ltd, etc.)
inno_df_missing = inno_df_missing.drop(columns=['uid'])
inno_df_missing = inno_df_missing.merge(df[['firm_name_clean', 'uid']], on=['firm_name_clean'], how='left')

inno_df_matched = pd.concat([inno_df_matched, inno_df_missing[~inno_df_missing.uid.isna()]])
inno_df_missing = inno_df_missing[inno_df_missing.uid.isna()].sort_values(['firm_name_norm'])

In [ ]:
inno_df_missing.to_csv(EXTERNAL_DATA_DIR / 'innosuisse_missing.csv', index=False)

In [ ]:
# Read the manually matched entries and combine them with the previous entries
manuell = pd.read_csv(EXTERNAL_DATA_DIR / 'innosuisse_missing copy.csv')

for col in ['start_date', 'end_date']:
    manuell[col] = pd.to_datetime(manuell[col])

manuell['uid'] = manuell['uid'].str.replace('-', '').str.replace('.', '')
inno_df_matched = pd.concat([inno_df_matched, manuell])

In [ ]:
# Split into uniquely matched and entries with multiple name+uid matches to check them manually
inno_duplicates = inno_df_matched[inno_df_matched.duplicated(subset=['project_id', 'firm_name_norm'], keep=False)].sort_values(['project_id', 'firm_name_norm', 'uid'])
inno_duplicates = inno_duplicates.drop_duplicates(subset=['project_id', 'uid'], keep='first')

inno_df_matched_unique = inno_df_matched[~inno_df_matched.duplicated(subset=['project_id', 'firm_name_norm'], keep=False)]

In [ ]:
inno_df_matched_unique.to_csv(EXTERNAL_DATA_DIR / 'innosuisse_unique_matches.csv', index=False)
inno_duplicates.to_csv(EXTERNAL_DATA_DIR / 'innosuisse_duplicates.csv', index=False)

In [ ]:
# Save the combined data with attached EHRAID and UID
df_final = pd.concat([inno_df_matched_unique, inno_duplicates]).drop_duplicates(subset=['project_id', 'uid'])
df_final = df_final.merge(df[~df['uid'].isna()][['uid', 'ehraid']].drop_duplicates(subset=['uid', 'ehraid']), on=['uid'], how='left')

cols = [col for col in df_final.columns if col not in ['uid', 'ehraid']] + ['ehraid', 'uid']
df_final = df_final[cols].sort_values('project_id')

df_final = df_final.drop(columns=['firm_name_norm', 'firm_name_combined', 'firm_name_clean'])

df_final.to_csv(PROCESSED_DATA_DIR / 'funding_data' / 'innosuisse_grants.csv', index=False)